# Projeto 1 - Recomendação de Produtos Locais - Disciplina de Introdução a Inteligência Artificial
Davi de Araujo Garcez Bueno - 211060586

Ana Luiza Campos Souza - 211055441

Célio Júnio de Freitas Eduardo - 211010350

# Lista de regiões (15 EMATER + extras)

Essa célula define as regiões do DF que serão utilizadas no sistema.
Incluímos as 15 regiões atendidas pela EMATER-DF e algumas regiões adicionais.

In [ ]:
import pandas as pd
import os
import datetime

regioes = [
    "Alexandre Gusmão, DF", "Brazlândia, DF", "Ceilândia, DF", "Gama, DF",
    "Jardim, DF", "PAD-DF, DF", "Paranoá, DF", "Pipiripau, DF", "Planaltina, DF",
    "São Sebastião, DF", "Sobradinho, DF", "Tabatinga, DF", "Taquara, DF", "Vargem Bonita, DF",
    "Santa Maria, DF", "Plano Piloto, DF", "Guará, DF", "Núcleo Bandeirante, DF",
    "Recanto das Emas, DF", "Samambaia, DF", "Taguatinga, DF"
]
df_regioes = pd.DataFrame(regioes, columns=["Região"])


# Geocodificação das regiões
Nessa célula utilizamos a biblioteca geopy para obter as coordenadas geográficas (latitude e longitude)
de cada região, permitindo sua visualização em mapas e cálculos de distância.

Então inicializamos o Nominatim com alguns ajustes, inicializando o geocodificador com um user-agent personalizado e usando o RateLimiter para evitar exceder limites de requisições da API.

Então usamos a geocodificação em cada região, iterando e extraindo a latitude e longitude.


In [3]:
# 1.2 – Geocodificação das regiões
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="ia_projeto")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df_regioes[["Lat", "Lon"]] = df_regioes["Região"].apply(
    lambda loc: pd.Series(geocode(loc)[1]) if geocode(loc) else pd.Series([None, None])
)

# Salva o resultado
os.makedirs("data", exist_ok=True)
df_regioes.to_csv("data/regioes_com_coordenadas.csv", index=False)


Nessa célula, definimos as 17 associações em uma lista.

In [ ]:
# 1.3 – Cadastro das 17 associações
associacoes = [
    {"Associação": "Afeca", "Regiões": "São Sebastião"},
    {"Associação": "Agrifam", "Regiões": "Taguatinga, Gama, Santa Maria"},
    {"Associação": "Amista", "Regiões": "Santa Maria"},
    {"Associação": "Aspaf", "Regiões": "Guará, Núcleo Bandeirante, Plano Piloto"},
    {"Associação": "AsSpag", "Regiões": "Brazlândia"},
    {"Associação": "Asphor", "Regiões": "Gama, Santa Maria, Plano Piloto"},
    {"Associação": "Asproc", "Regiões": "Ceilândia, Recanto das Emas, Samambaia, Paranoá, Planaltina"},
    {"Associação": "Aspronte", "Regiões": "Ceilândia, Recanto das Emas"},
    {"Associação": "Astraf", "Regiões": "Guará"},
    {"Associação": "Coopbrasil", "Regiões": "Gama, Recanto das Emas, Samambaia, Núcleo Bandeirante, Planaltina, Brazlândia, Ceilândia"},
    {"Associação": "CooperHorti", "Regiões": "Paranoá"},
    {"Associação": "Prorural", "Regiões": "Plano Piloto, Paranoá"},
    {"Associação": "Coopebraz", "Regiões": "Brazlândia, Taguatinga, Samambaia, Recanto das Emas"},
    {"Associação": "Coopermista", "Regiões": "Planaltina"},
    {"Associação": "Rede Terra", "Regiões": "Santa Maria"},
    {"Associação": "Cootaquara", "Regiões": "Ceilândia"},
    {"Associação": "Cooperbrasília", "Regiões": "Sobradinho, São Sebastião"},
]
df_assoc = pd.DataFrame(associacoes)


# Expandir por região

Nessa célula, expandimos o DataFrame de associações para ter uma linha para cada combinação de associação e região, facilitando a visualização e o cálculo das distâncias.

Então damos um Merge com as coordenadas para adicionar a latitude e longitude a cada associação-região.

In [5]:
# 1.4 – Expandir por região
linhas_expandidas = []
for _, row in df_assoc.iterrows():
    regioes = [r.strip() + ", DF" for r in row["Regiões"].split(",")]
    for reg in regioes:
        linhas_expandidas.append({
            "Associação": row["Associação"],
            "Região": reg
        })

df_assoc_exp = pd.DataFrame(linhas_expandidas)

# Merge com coordenadas
df_assoc_exp = df_assoc_exp.drop(columns=["Lat", "Lon"], errors="ignore")
df_assoc_exp = df_assoc_exp.merge(df_regioes, on="Região", how="left")
df_assoc_exp = df_assoc_exp.dropna(subset=["Lat", "Lon"])
df_assoc_exp.to_csv("data/associacoes_formatadas.csv", index=False)


# Cálculo de distância

Esta célula implementa o cálculo de distância entre a localização do usuário e cada associação, permitindo recomendar as associações mais próximas.

Então definimos a posição do usuário no Plano Piloto como exemplo e usamos uma função para calcular a distância em quilometros entre o usuário e cada associação.

Aplicamos então a função para cada associação e depois ordenamos por distância para mostrar as 10 associações mais próximas.



In [7]:
# Etapa 3 – Calcular distância até o usuário
from geopy.distance import geodesic

user_location = (-15.7939, -47.8828)  # Exemplo: Plano Piloto

def calcular_distancia(lat, lon):
    return geodesic(user_location, (lat, lon)).km

df_assoc_exp["Distância_km"] = df_assoc_exp.apply(
    lambda row: calcular_distancia(row["Lat"], row["Lon"]), axis=1
)

# Mostrar as mais próximas
df_assoc_exp = df_assoc_exp.sort_values("Distância_km")
df_assoc_exp[["Associação", "Região", "Distância_km"]].head(10)


,Associação,Região,Distância_km
5,Aspaf,"Guará, DF",10.593852
19,Astraf,"Guará, DF",10.593852
27,CooperHorti,"Paranoá, DF",11.227612
15,Asproc,"Paranoá, DF",11.227612
29,Proural,"Paranoá, DF",11.227612
23,Coopbrasil,"Núcleo Bandeirante, DF",12.507099
6,Aspaf,"Núcleo Bandeirante, DF",12.507099
7,Aspaf,"Plano Piloto, DF",14.645694
11,Asphor,"Plano Piloto, DF",14.645694
28,Proural,"Plano Piloto, DF",14.645694
